In [1]:
# %matplotlib inline

# Sujan Graph

### 1. Create latest Graph

From `relationships.csv` file,  we create latest graph html file for maximum year. We create this file for purpose that would be clear in next step. 

Any one could see this generated html file with interactive graph. This is not efficient. Also voila forbids loading the file..

In [2]:
from pyvis.network import Network
import pandas as pd
import networkx as nx
from ipywidgets import interact, IntSlider, fixed, Layout
from IPython.display import IFrame, display, Javascript

# Function to read the CSV and return a DataFrame
def read_csv_to_df(csv_file_path):
    df = pd.read_csv(csv_file_path)
    return df

def filter_graph_by_year(df, year):
    # Filter DataFrame for relationships up to the selected year
    df_filtered = df[df['year'] <= year]

    # Recreate the graph based on the filtered DataFrame
    G_filtered = nx.Graph()

    # Calculate weights for persons and companies based on their frequency up to the selected year
    person_weights = df_filtered['person'].value_counts().to_dict()
    company_weights = df_filtered['company'].value_counts().to_dict()

    # Add nodes and edges based on the filtered DataFrame
    for _, row in df_filtered.iterrows():
        person_weight = person_weights.get(row['person'], 1)
        company_weight = company_weights.get(row['company'], 1)

        G_filtered.add_node(row['person'], type='person', weight=person_weight)
        G_filtered.add_node(row['company'], type='company', weight=company_weight)
        G_filtered.add_edge(row['person'], row['company'], relation=row['year'])

    return G_filtered


def plot_networkx_graph_by_year_pyvis(df, year):
    G = filter_graph_by_year(df, year)
    
    # Create a Pyvis network graph
    net = Network(notebook=True, height="400px", width="100%", cdn_resources="remote")
    

    # Add nodes with initial properties
    for node, attr in G.nodes(data=True):
        color = 'violet' if attr['type'] == 'person' else 'orange'
        value = attr.get('weight', 1)
        # Check for specific name to change the color
        if "sujan" in node.lower() and attr['type'] == 'person':
            color = '#3d46ff'
        net.add_node(node, title=node, color=color, value=value)


    # Add edges to the network
    for source, target, edge_attrs in G.edges(data=True):
        net.add_edge(source, target, title=edge_attrs['relation'])
    
    
    # Set options for the visualization
    net.set_options("""
    var options = {
        "physics": {
            "enabled": true,
            "minVelocity": 0.75,
            "repulsion": {
                "centralGravity": 0.1,
                "springLength": 200,
                "springConstant": 0.05,
                "nodeDistance": 300,
                "damping": 0.5
            },
            "forceAtlas2Based": {
                "avoidOverlap": 1,
                "gravitationalConstant": -50,
                "centralGravity": 0.01,
                "springConstant": 0.08,
                "damping": 0.4
            },
            "maxVelocity": 50,
            "solver": "forceAtlas2Based",
            "timestep": 0.35
        }
    }
    """)
    
    # Generate the network graph
    net.show('sujan_latest.html')
    return IFrame('sujan_latest.html', width='100%', height='450px')

# Assuming `read_csv_to_df` is a function you have defined to read the CSV file into a DataFrame.
df = read_csv_to_df('relationships.csv')
year_max = df['year'].max()
plot_networkx_graph_by_year_pyvis(df, year_max)
print("Latest graph generated successfully")

Latest graph generated successfully


### 2. Extract Positions

The file created earlier using pyvis has best positioned nodes. We extract this for our custom node positions in next step, where for every year we want to see the graph. If we do not hard code best node position, for every year, pyvis decides its own positions, which make tracking the yearly changes very difficult visually. 

To achieve this, we use hints from [here](https://github.com/WestHealth/pyvis/issues/157#issuecomment-1288566121) to extract and dump the positions, and read that file directly in our next step. 

```js
function download(content, fileName, contentType) {
    var a = document.createElement("a");
    var file = new Blob([content], {type: contentType});
    a.href = URL.createObjectURL(file);
    a.download = fileName;
    a.click();
}
network.storePositions();
download(JSON.stringify(data.nodes.get()), 'positions.txt', 'text/plain');
```

### Create Yearly Evolution Graph

Now that we have best positions from file for nodes, for every year, for matching nodes, we create graph at those positions only as below. Sometimes, output does not show up here, but it will in voila server. 

The change from previous year and shown year, would be displayed in yellow edges, so it would be clear what changed every year. This is one important visual hint to get to know what delta happened in an year.  

In [3]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from ipywidgets import interact, IntSlider, Layout
from IPython.display import display, HTML
import json

def read_positions(file_path):
    with open(file_path, "r") as file:
        positions_data = json.load(file)
    positions = {item['id']: (item['x'], item['y']) for item in positions_data}
    return positions

def prepare_full_graph(df):
    G = nx.Graph()
    person_weights = df['person'].value_counts().to_dict()
    company_weights = df['company'].value_counts().to_dict()

    for _, row in df.iterrows():
        person_weight = person_weights.get(row['person'], 1)
        company_weight = company_weights.get(row['company'], 1)
        G.add_node(row['person'], type='person', year=row['year'], label=row['person'], weight=person_weight)
        G.add_node(row['company'], type='company', year=row['year'], label=row['company'], weight=company_weight)
        G.add_edge(row['person'], row['company'], year=row['year'])

    return G

def plot_networkx_graph_by_year_pyvis_2(positions, G, year, prev_year_edges=set()):
    net = Network(notebook=True, height="450px", width="100%", bgcolor="#222222", font_color="white", cdn_resources='remote')
    current_edges = set()

    for node, attr in G.nodes(data=True):
        if attr['year'] <= year:
            color = 'violet' if attr['type'] == 'person' else 'orange'
            value = attr.get('weight', 1)
            if "sujan" in node.lower() and attr['type'] == 'person':
                color = '#87dfff'
            node_pos = positions.get(node, (0, 0))
            net.add_node(node, label=node, x=node_pos[0], y=node_pos[1], color=color, value=value, fixed=True)

    for source, target, edge_attrs in G.edges(data=True):
        if edge_attrs['year'] <= year:
            current_edges.add((source, target))
            if (source, target) not in prev_year_edges and edge_attrs['year'] > year - 1:
                edge_color = 'yellow'
                width = 5  # 2 times thicker
            else:
                edge_color = '#87dfff'
                width = 1
            net.add_edge(source, target, color=edge_color, width=width)

    net.show("tmp3.html")
    display(HTML("tmp3.html"))
    return current_edges

def interactive_networkx_graph_pyvis(G, positions):
    year_min, year_max = min(attr['year'] for _, attr in G.nodes(data=True)), max(attr['year'] for _, attr in G.nodes(data=True))
    prev_year_edges = set()

    def update_graph(year):
        nonlocal prev_year_edges
        prev_year_edges = plot_networkx_graph_by_year_pyvis_2(positions, G, year, prev_year_edges)

    interact(update_graph, year=IntSlider(value=year_min, min=year_min, max=year_max, step=1, description='Year:', layout=Layout(width='100%')))

# Load data and create graph
df = pd.read_csv('relationships.csv')  # Replace with your actual CSV data path
positions = read_positions('positions.txt')  # Use the path where the positions file is saved
G = prepare_full_graph(df)
interactive_networkx_graph_pyvis(G, positions)


interactive(children=(IntSlider(value=1994, description='Year:', layout=Layout(width='100%'), max=2023, min=19…